# Affine Transformation

Affine Transformationen sind Transformationen des Raums, bei denen bestimmte geometrische Eigenschaften erhalten bleiben.
Lose formuliert:

- Punkte einer Geraden werden wieder auf eine Gerade abgebildet
- Parallele Geraden bleiben parallel
- Das Verhältnis dreier Punkte einer Geraden bleibt erhalten

Affine Transformationen kann man mittels Matrixmultiplikation darstellen.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def plot(X, y, fig, ax):
    ax.scatter(X[:,0], X[:,1], c=y, alpha=0.5)
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.axis('equal');


Erzeugen wir uns zunächst mal eine Punktwolke:

In [ ]:
m = 200

rng = np.random.default_rng(50)
distr = rng.standard_normal(size=(2,m))

X_id = np.dot(np.identity(2), distr).T
y_id = np.ndarray([m,1]).fill(0)

fig, ax = plt.subplots(1, figsize=(10,10))
plot(X_id, y_id, fig, ax);


Die Punktwolke können wir durch Multiplikation mit einer 2x2 Matrix verzerren:

In [ ]:

theta = np.pi/2
rot = [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
shear = [[1, 2], [0, 1]]
stretchX = [[2, 0],[0, 1]]
stretchY = [[1, 0],[0, 2]]

fig, axs = plt.subplots(2,3, figsize=(15,10))

def plotTransformed(label, input, transformation, ax):
    box = np.asarray([[x,y] for x in np.linspace(-4,4, 20) for y in np.linspace(-4,4, 20)]).T
    box_transformed = np.dot(transformation, box).T
    data = np.dot(transformation, input).T
    A = data[:, 0]
    B = data[:, 1]
    ax.axis('equal')
    ax.set(xlim=[-10,10], ylim=[-10, 10])
    ax.set_title(label)
    ax.scatter(A, B, alpha=0.5);
    ax.scatter(box_transformed[:,0], box_transformed[:,1], s=5, c='grey', alpha=0.1);

plotTransformed('Identität', distr, np.identity(2), ax=axs[0, 0])
plotTransformed('Rotation', distr, rot, ax=axs[0, 1])
plotTransformed('Shear', distr, shear, ax=axs[0, 2])
plotTransformed('Stretch X', distr, stretchX, ax=axs[1, 0])
plotTransformed('Stretch Y', distr, stretchY, ax=axs[1, 1])
plotTransformed('Combined', distr, np.dot(np.dot(stretchX, stretchY), np.dot(rot, shear)), ax=axs[1, 2])


Jetzt ermitteln wir eine zufällige Matrix, mit der wir diese Wolke verzerren:

In [ ]:
S = rng.random(size=(2,2))
print(f'S = {S}')

In [ ]:
X = np.dot(S, distr).T
y = np.ndarray([m,1]).fill(1)

fig, ax = plt.subplots(1, figsize=(10,10))
plotTransformed('Verzerrte Punktwolke', distr, S, ax=ax);
ax.set(xlim=[-2.5, 2.5], ylim=[-2.5, 2.5])

PCA - Principal Component Analysis - ist eine Methode, um die maßgeblichen "Komponenten" in einer Datenmenge herauszufinden.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2, whiten=True)
pca.fit(X)

Die Ergebnisse von PCA

In [ ]:
print(f'Components ( = Eigenvektoren):\n {pca.components_}')
print(f'\nVarianz ( = Eigenwerte ):\n {pca.explained_variance_}')


Schauen wir uns mal die Vektoren an, die durch PCA definiert sind:

In [ ]:
def draw_vector(v0, v1, ax):
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v, ax)
fig

D.h. die durch PCA definierten Vektoren, beschreiben das Koordinatensystem, welches der Verteilung der Punktwolke folgt.

Wir können die Punktwolke auch als Punkte im durch PCA definierten Koordinatensystem darstellen:

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(21,7))

# zunächst noch mal unsere Punktewolke
plot(X, y, fig, axs[0]);
axs[0].set_title('Verzerrte Punktwolke');
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v, axs[0])

# und jetzt die Transformation durch PCA
X_pca = pca.transform(X)

axs[1].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
draw_vector([0, 0], [0, 3], ax=axs[1])
draw_vector([0, 0], [3, 0], ax=axs[1])
axs[1].axis('equal')
axs[1].set(xlabel='component 1', ylabel='component 2',
          title='principal components')
